### Rolling window estimation schemes

In [1]:
import os
os.getcwd()
# os.chdir(path)    # or you can set your working dir.

'/Users/xingfuxu/PycharmProjects/EquityPremiumPredictionML-Jupyter'

In [2]:
# Your working dir should include "NN_models.py", Perform_CW_test.py" and "Perform_PT_test.py" files.
from Perform_CW_test import CW_test
from Perform_PT_test import PT_test
from NN_models import Net2, Net4

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
import torch
from skorch import NeuralNetRegressor
from tqdm import tqdm
#
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
#
import warnings

In [4]:
# set seed
torch.manual_seed(1)
np.random.seed(1)

# read data
predictor_df = pd.read_excel(open('ml_equity_premium_data.xlsx', 'rb'), sheet_name='result_predictor')
predictor_df.head()

,month,log_equity_premium,equity_premium,DP,DY,EP,SVAR,BM,NTIS,TBL,...,MA_2_9,MA_2_12,MA_3_9,MA_3_12,MOM_1,MOM_2,MOM_3,MOM_6,MOM_9,MOM_12
0,192701,-0.005710,-0.00571,-2.942374,-2.963349,-2.374773,0.00047,0.44371,0.05082,3.23,...,1,1,1,1,0,0,1,1,1,1
1,192702,0.042017,0.04302,-2.979535,-2.932946,-2.430353,0.00029,0.42850,0.05167,3.29,...,1,1,1,1,1,1,1,1,1,1
2,192703,0.004697,0.00472,-2.976535,-2.970053,-2.445079,0.00092,0.46977,0.04636,3.20,...,1,1,1,1,1,1,1,1,1,1
3,192704,0.009940,0.01002,-2.984225,-2.967143,-2.471309,0.00060,0.45675,0.05051,3.39,...,1,1,1,1,1,1,1,1,1,1
4,192705,0.057987,0.05985,-3.025963,-2.975058,-2.531446,0.00039,0.43478,0.05528,3.33,...,1,1,1,1,1,1,1,1,1,1


In [5]:
# remove irrelavent columns
predictor0 = predictor_df.drop(['month', 'equity_premium'], axis=1)
# get all the predictors and set the log equity premium 1-month ahead
predictor = np.concatenate([predictor0['log_equity_premium'][1:].values.reshape(-1, 1),
                            predictor0.iloc[0:(predictor0.shape[0] - 1), 1:]], axis=1)

# number of rows
N = predictor.shape[0]

# number of all columns, including the log equity premium
n_cols = predictor.shape[1]

# Actual one-month ahead log equity premium
actual = predictor[:, [0]]

# Historical average forecasting as benchmark
y_pred_HA = predictor0['log_equity_premium'].values[0:(predictor0.shape[0] - 1), ].cumsum() / np.arange(1, N + 1)
y_pred_HA = y_pred_HA.reshape(-1, 1)

# Compute out-of-sample R-squared
def compute_oos_r_square(actual, y_benchmark, y_pred):
    MSFE_benchmark = mean_squared_error(y_benchmark, actual)
    MSFE_pred = mean_squared_error(y_pred, actual)
    return 1 - MSFE_pred / MSFE_benchmark

### Five years rolling estimation scheme

In [6]:
## Out-of-sample: 1957:01-2020:12
in_out_1957 = predictor_df.index[predictor_df['month'] == 195701][0]
actual_1957 = actual[in_out_1957:, ]
y_pred_HA_1957 = y_pred_HA[in_out_1957:, ]
MSFE_HA_1957 = mean_squared_error(y_pred_HA_1957, actual_1957)

# Machine Learning methods used in GKX (2020)
y_pred_PLS_1957, y_pred_PCR_1957,  y_pred_LASSO_1957 = [], [], []
y_pred_ENet_1957, y_pred_RF_1957 = [], []
y_pred_NN2_1957, y_pred_NN4_1957 = [], []

## Other commonly used machine learning method
y_pred_Ridge_1957 = []

# control the update month of models during out-of-sample period. 
month_index = 1  # We update our models annually, meaning we refresh them in months 1, 13, 25, ...


for t in tqdm(range(in_out_1957, N)):
    #
    warnings.filterwarnings('ignore')
    X_train_all = predictor[(t - 5 * 12):t, 1:n_cols]
    y_train_all = predictor[(t - 5 * 12):t, 0]
    # set 15% of all the train data as validation set
    X_train = X_train_all[0:int(len(X_train_all) * 0.85), :]
    X_validation = X_train_all[int(len(X_train_all) * 0.85):t, :]
    y_train = y_train_all[0:int(len(X_train_all) * 0.85)]
    y_validation = y_train_all[int(len(X_train_all) * 0.85):t]
    #
    if month_index % 12 == 1:
        month_index += 1

        # PLS
        PLS_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PLS_result = {}
        for param in ParameterGrid(PLS_param):
            PLS = PLSRegression(**param)
            PLS.fit(X_train, y_train)
            mse = mean_squared_error(PLS.predict(X_validation), y_validation)
            PLS_result[str(param)] = mse

        PLS_best_param = eval(min(PLS_result, key=PLS_result.get))
        PLS_model = PLSRegression(**PLS_best_param)
        PLS_model.fit(X_train_all, y_train_all)
        y_pred_PLS_1957.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])

        # PCR
        PCR_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PCR_result = {}
        for param in ParameterGrid(PCR_param):
            pca = PCA(**param)
            pca.fit(X_train)
            comps = pca.transform(X_train)
            forecast = LinearRegression()
            forecast.fit(comps, y_train)
            mse = mean_squared_error(forecast.predict(pca.transform(X_validation)), y_validation)
            PCR_result[str(param)] = mse
        #
        PCR_best_param = eval(min(PCR_result, key=PCR_result.get))
        #
        PCR_model = PCA(**PCR_best_param)
        PCR_model.fit(X_train_all)
        PCR_comps = PCR_model.transform(X_train_all)
        PCR_forecast = LinearRegression()
        PCR_forecast.fit(PCR_comps, y_train_all)
        y_pred_PCR_1957.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])

        # LASSO
        LASSO_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2))}
        LASSO_result = {}
        for param in ParameterGrid(LASSO_param):
            LASSO = Lasso(**param)
            LASSO.fit(X_train, y_train)
            mse = mean_squared_error(LASSO.predict(X_validation), y_validation)
            LASSO_result[str(param)] = mse
        #
        LASSO_best_param = eval(min(LASSO_result, key=LASSO_result.get))
        #
        LASSO_model = Lasso(**LASSO_best_param)
        LASSO_model.fit(X_train_all, y_train_all)
        y_pred_LASSO_1957.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])

        # ENet
        ENet_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2)),
                      'l1_ratio': list(np.arange(0.2, 1, 0.3))}
        ENet_result = {}
        for param in ParameterGrid(ENet_param):
            ENet = ElasticNet(**param, tol=1e-2)
            ENet.fit(X_train, y_train)
            mse = mean_squared_error(ENet.predict(X_validation), y_validation)
            ENet_result[str(param)] = mse

        ENet_best_param = eval(min(ENet_result, key=ENet_result.get))
        ENet_model = ElasticNet(**ENet_best_param, tol=1e-2)
        ENet_model.fit(X_train_all, y_train_all)
        y_pred_ENet_1957.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])


        # RF
        RF_param = {'n_estimators': [10, 50, 100, 150, 200],
                    'max_depth': [2, 3, 4],
                    'min_samples_leaf': [1, 3, 5]}
        RF_result = {}
        for param in ParameterGrid(RF_param):
            RF = RandomForestRegressor(**param)
            RF.fit(X_train, y_train)
            mse = mean_squared_error(RF.predict(X_validation), y_validation)
            RF_result[str(param)] = mse

        RF_best_param = eval(min(RF_result, key=RF_result.get))
        RF_model = RandomForestRegressor(**RF_best_param)
        RF_model.fit(X_train_all, y_train_all)
        y_pred_RF_1957.append(RF_model.predict(predictor[[t], 1:n_cols])[0])

        # Neural Network Models: NN2 & NN4
        X_train_all_tensor = torch.tensor(X_train_all, dtype=torch.float)
        y_train_all_tensor = torch.tensor(y_train_all.reshape(-1, 1), dtype=torch.float)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float)
        y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float)
        X_validation_tensor = torch.tensor(X_validation, dtype=torch.float)
        y_validation_tensor = torch.tensor(y_validation.reshape(-1, 1), dtype=torch.float)

        # NN2
        NN2_result = {}
        NN2_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_output": 1}
        NN2_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                    'lr': [0.001, 0.01],
                    'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN2_param):
            NN2 = NeuralNetRegressor(Net2, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN2_architecture, **param)
            NN2.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN2.predict(X_validation_tensor), y_validation)
            NN2_result[str(param)] = mse
        
        #
        NN2_best_param = eval(min(NN2_result, key=NN2_result.get))
        NN2_model = NeuralNetRegressor(Net2, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN2_architecture, **NN2_best_param)
        NN2_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN2_1957.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        #


        # NN4
        NN4_result = {}
        NN4_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_hidden3": 8,  "module__n_hidden4": 4,
                            "module__n_output": 1}
        NN4_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                     'lr': [0.001, 0.01],
                     'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN4_param):
            NN4 = NeuralNetRegressor(Net4, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN4_architecture, **param)
            NN4.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN4.predict(X_validation_tensor), y_validation)
            NN4_result[str(param)] = mse

        #
        NN4_best_param = eval(min(NN4_result, key=NN4_result.get))
        NN4_model = NeuralNetRegressor(Net4, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN4_architecture, **NN4_best_param)
        NN4_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN4_1957.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])


        ## Other commmonly used ML methods
        # Ridge
        Ridge_param = {'alpha': list(10 ** np.arange(0, 20 + 0.001, 1))}
        Ridge_result = {}
        for param in ParameterGrid(Ridge_param):
            RIDGE = Ridge(**param)
            RIDGE.fit(X_train, y_train)
            mse = mean_squared_error(RIDGE.predict(X_validation), y_validation)
            Ridge_result[str(param)] = mse
        #
        Ridge_best_param = eval(min(Ridge_result, key=Ridge_result.get))
        Ridge_model = Ridge(**Ridge_best_param)
        Ridge_model.fit(X_train_all, y_train_all)
        y_pred_Ridge_1957.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

    else:
        month_index += 1
        y_pred_PLS_1957.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])
        y_pred_PCR_1957.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])
        y_pred_LASSO_1957.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_ENet_1957.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_RF_1957.append(RF_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_NN2_1957.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        y_pred_NN4_1957.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        # Other commonly used ML methods
        y_pred_Ridge_1957.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

        

        
y_ml_pred_1957 = pd.DataFrame(np.array([y_pred_PLS_1957, y_pred_PCR_1957, y_pred_LASSO_1957,
                           y_pred_ENet_1957, y_pred_RF_1957, y_pred_NN2_1957,  
                           y_pred_NN4_1957, y_pred_Ridge_1957]),
                 index=['PLS', 'PCR', 'LASSO', 'ENet', 
                        'RF', 'NN2', 'NN4', 'Ridge'],
                 columns=predictor_df.month[in_out_1957:N]).T

# Performance compared with HA benchmark

ml_oos_performance_5years = []

for col in y_ml_pred_1957.columns:
    oos_r_square = compute_oos_r_square(actual_1957, y_pred_HA_1957, y_ml_pred_1957[[col]].to_numpy())
    MSFE_adjusted, pvalue_MSFE = CW_test(actual_1957, y_pred_HA_1957, y_ml_pred_1957[[col]].to_numpy())
    success_ratio, PT_stat, pvalue_PT = PT_test(actual_1957, y_ml_pred_1957[[col]].to_numpy())
    ml_oos_performance_5years.append([oos_r_square * 100, MSFE_adjusted, pvalue_MSFE, success_ratio * 100, PT_stat, pvalue_PT])


ml_oos_performance_df_5years = pd.DataFrame(np.array(ml_oos_performance_5years),
                                          index=y_ml_pred_1957.columns,
                                          columns=['oos_r_square', 'MSFE_adjusted', 'pvalue_MSFE',
                                                   'success_ratio', 'PT_stat', 'pvalue_PT'])
# success ratio of HA
success_ratio_HA_1957, PT_HA_1957, p2_HA_1957 = PT_test(actual_1957, y_pred_HA_1957)
ml_oos_performance_df_5years.loc['HA'] = [0, np.nan, np.nan, success_ratio_HA_1957 * 100, PT_HA_1957, p2_HA_1957]
ml_oos_performance_df_5years

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 767/767 [25:36<00:00,  2.00s/it]


,oos_r_square,MSFE_adjusted,pvalue_MSFE,success_ratio,PT_stat,pvalue_PT
PLS,-47.239596,1.083377,0.139321,52.411995,-0.164357,0.565275
PCR,-11.831484,0.818648,0.206494,56.062581,1.013248,0.155471
LASSO,-82.461558,-0.003547,0.501415,57.105606,1.118965,0.131578
ENet,-74.396652,0.606137,0.272212,55.410691,0.585414,0.279135
RF,-16.913349,0.194882,0.422743,52.542373,-0.226891,0.589746
NN2,-102.982502,-0.304749,0.619721,46.675359,-2.474727,0.993333
NN4,-358.211154,-0.810580,0.791197,51.499348,-1.006715,0.842964
Ridge,-26.633373,0.143339,0.443011,56.453716,0.790713,0.214556
HA,0.000000,NaN,NaN,59.973924,NaN,NaN


### Ten years rolling window estimation scheme

In [7]:
## Out-of-sample: 1957:01-2020:12
in_out_1957 = predictor_df.index[predictor_df['month'] == 195701][0]
actual_1957 = actual[in_out_1957:, ]
y_pred_HA_1957 = y_pred_HA[in_out_1957:, ]
MSFE_HA_1957 = mean_squared_error(y_pred_HA_1957, actual_1957)

# Machine Learning methods used in GKX (2020)
y_pred_PLS_1957, y_pred_PCR_1957,  y_pred_LASSO_1957 = [], [], []
y_pred_ENet_1957, y_pred_RF_1957 = [], []
y_pred_NN2_1957, y_pred_NN4_1957 = [], []

## Other commonly used machine learning method
y_pred_Ridge_1957 = []

# control the update month of models during out-of-sample period. 
month_index = 1  # We update our models annually, meaning we refresh them in months 1, 13, 25, ...


for t in tqdm(range(in_out_1957, N)):
    #
    X_train_all = predictor[(t - 10 * 12):t, 1:n_cols]
    y_train_all = predictor[(t - 10 * 12):t, 0]
    # set 15% of all the train data as validation set
    X_train = X_train_all[0:int(len(X_train_all) * 0.85), :]
    X_validation = X_train_all[int(len(X_train_all) * 0.85):t, :]
    y_train = y_train_all[0:int(len(X_train_all) * 0.85)]
    y_validation = y_train_all[int(len(X_train_all) * 0.85):t]
    #
    if month_index % 12 == 1:
        month_index += 1

        # PLS
        PLS_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PLS_result = {}
        for param in ParameterGrid(PLS_param):
            PLS = PLSRegression(**param)
            PLS.fit(X_train, y_train)
            mse = mean_squared_error(PLS.predict(X_validation), y_validation)
            PLS_result[str(param)] = mse

        PLS_best_param = eval(min(PLS_result, key=PLS_result.get))
        PLS_model = PLSRegression(**PLS_best_param)
        PLS_model.fit(X_train_all, y_train_all)
        y_pred_PLS_1957.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])

        # PCR
        PCR_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PCR_result = {}
        for param in ParameterGrid(PCR_param):
            pca = PCA(**param)
            pca.fit(X_train)
            comps = pca.transform(X_train)
            forecast = LinearRegression()
            forecast.fit(comps, y_train)
            mse = mean_squared_error(forecast.predict(pca.transform(X_validation)), y_validation)
            PCR_result[str(param)] = mse
        #
        PCR_best_param = eval(min(PCR_result, key=PCR_result.get))
        #
        PCR_model = PCA(**PCR_best_param)
        PCR_model.fit(X_train_all)
        PCR_comps = PCR_model.transform(X_train_all)
        PCR_forecast = LinearRegression()
        PCR_forecast.fit(PCR_comps, y_train_all)
        y_pred_PCR_1957.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])

        # LASSO
        LASSO_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2))}
        LASSO_result = {}
        for param in ParameterGrid(LASSO_param):
            LASSO = Lasso(**param)
            LASSO.fit(X_train, y_train)
            mse = mean_squared_error(LASSO.predict(X_validation), y_validation)
            LASSO_result[str(param)] = mse
        #
        LASSO_best_param = eval(min(LASSO_result, key=LASSO_result.get))
        #
        LASSO_model = Lasso(**LASSO_best_param)
        LASSO_model.fit(X_train_all, y_train_all)
        y_pred_LASSO_1957.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])

        # ENet
        ENet_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2)),
                      'l1_ratio': list(np.arange(0.2, 1, 0.3))}
        ENet_result = {}
        for param in ParameterGrid(ENet_param):
            ENet = ElasticNet(**param)
            ENet.fit(X_train, y_train)
            mse = mean_squared_error(ENet.predict(X_validation), y_validation)
            ENet_result[str(param)] = mse

        ENet_best_param = eval(min(ENet_result, key=ENet_result.get))
        ENet_model = ElasticNet(**ENet_best_param)
        ENet_model.fit(X_train_all, y_train_all)
        y_pred_ENet_1957.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])


        # RF
        RF_param = {'n_estimators': [10, 50, 100, 150, 200],
                    'max_depth': [2, 3, 4],
                    'min_samples_leaf': [1, 3, 5]}
        RF_result = {}
        for param in ParameterGrid(RF_param):
            RF = RandomForestRegressor(**param)
            RF.fit(X_train, y_train)
            mse = mean_squared_error(RF.predict(X_validation), y_validation)
            RF_result[str(param)] = mse

        RF_best_param = eval(min(RF_result, key=RF_result.get))
        RF_model = RandomForestRegressor(**RF_best_param)
        RF_model.fit(X_train_all, y_train_all)
        y_pred_RF_1957.append(RF_model.predict(predictor[[t], 1:n_cols])[0])

        # Neural Network Models: NN2 & NN4
        X_train_all_tensor = torch.tensor(X_train_all, dtype=torch.float)
        y_train_all_tensor = torch.tensor(y_train_all.reshape(-1, 1), dtype=torch.float)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float)
        y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float)
        X_validation_tensor = torch.tensor(X_validation, dtype=torch.float)
        y_validation_tensor = torch.tensor(y_validation.reshape(-1, 1), dtype=torch.float)

        # NN2
        NN2_result = {}
        NN2_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_output": 1}
        NN2_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                    'lr': [0.001, 0.01],
                    'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN2_param):
            NN2 = NeuralNetRegressor(Net2, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN2_architecture, **param)
            NN2.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN2.predict(X_validation_tensor), y_validation)
            NN2_result[str(param)] = mse
        
        #
        NN2_best_param = eval(min(NN2_result, key=NN2_result.get))
        NN2_model = NeuralNetRegressor(Net2, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN2_architecture, **NN2_best_param)
        NN2_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN2_1957.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        #


        # NN4
        NN4_result = {}
        NN4_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_hidden3": 8,  "module__n_hidden4": 4,
                            "module__n_output": 1}
        NN4_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                     'lr': [0.001, 0.01],
                     'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN4_param):
            NN4 = NeuralNetRegressor(Net4, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN4_architecture, **param)
            NN4.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN4.predict(X_validation_tensor), y_validation)
            NN4_result[str(param)] = mse

        #
        NN4_best_param = eval(min(NN4_result, key=NN4_result.get))
        NN4_model = NeuralNetRegressor(Net4, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN4_architecture, **NN4_best_param)
        NN4_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN4_1957.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])


        ## Other commmonly used ML methods
        # Ridge
        Ridge_param = {'alpha': list(10 ** np.arange(0, 20 + 0.001, 1))}
        Ridge_result = {}
        for param in ParameterGrid(Ridge_param):
            RIDGE = Ridge(**param)
            RIDGE.fit(X_train, y_train)
            mse = mean_squared_error(RIDGE.predict(X_validation), y_validation)
            Ridge_result[str(param)] = mse
        #
        Ridge_best_param = eval(min(Ridge_result, key=Ridge_result.get))
        Ridge_model = Ridge(**Ridge_best_param)
        Ridge_model.fit(X_train_all, y_train_all)
        y_pred_Ridge_1957.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

    else:
        month_index += 1
        y_pred_PLS_1957.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])
        y_pred_PCR_1957.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])
        y_pred_LASSO_1957.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_ENet_1957.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_RF_1957.append(RF_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_NN2_1957.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        y_pred_NN4_1957.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        # Other commonly used ML methods
        y_pred_Ridge_1957.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

        

        
y_ml_pred_1957 = pd.DataFrame(np.array([y_pred_PLS_1957, y_pred_PCR_1957, y_pred_LASSO_1957,
                           y_pred_ENet_1957, y_pred_RF_1957, y_pred_NN2_1957,  
                           y_pred_NN4_1957, y_pred_Ridge_1957]),
                 index=['PLS', 'PCR', 'LASSO', 'ENet', 
                        'RF', 'NN2', 'NN4', 'Ridge'],
                 columns=predictor_df.month[in_out_1957:N]).T

# Performance compared with HA benchmark

ml_oos_performance_10years = []

for col in y_ml_pred_1957.columns:
    oos_r_square = compute_oos_r_square(actual_1957, y_pred_HA_1957, y_ml_pred_1957[[col]].to_numpy())
    MSFE_adjusted, pvalue_MSFE = CW_test(actual_1957, y_pred_HA_1957, y_ml_pred_1957[[col]].to_numpy())
    success_ratio, PT_stat, pvalue_PT = PT_test(actual_1957, y_ml_pred_1957[[col]].to_numpy())
    ml_oos_performance_10years.append([oos_r_square * 100, MSFE_adjusted, pvalue_MSFE, success_ratio * 100, PT_stat, pvalue_PT])


ml_oos_performance_df_10years = pd.DataFrame(np.array(ml_oos_performance_10years),
                                          index=y_ml_pred_1957.columns,
                                          columns=['oos_r_square', 'MSFE_adjusted', 'pvalue_MSFE',
                                                   'success_ratio', 'PT_stat', 'pvalue_PT'])
ml_oos_performance_df_10years.loc['HA'] = [0, np.nan, np.nan, success_ratio_HA_1957 * 100, PT_HA_1957, p2_HA_1957]
ml_oos_performance_df_10years

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 767/767 [35:01<00:00,  2.74s/it]


,oos_r_square,MSFE_adjusted,pvalue_MSFE,success_ratio,PT_stat,pvalue_PT
PLS,-15.893212,1.049371,0.147004,55.410691,1.356501,0.087470
PCR,-6.902878,0.899997,0.184061,57.757497,1.715461,0.043130
LASSO,-4.771582,1.970124,0.024412,59.843546,2.558406,0.005258
ENet,-5.346111,1.982762,0.023697,59.843546,2.747300,0.003004
RF,-14.836244,0.534075,0.296645,51.368970,-0.222107,0.587885
NN2,-100.028150,0.707580,0.239603,52.542373,0.229788,0.409128
NN4,-103.312912,-1.065146,0.856595,55.541069,1.161620,0.122695
Ridge,-5.562798,1.265462,0.102853,58.930900,2.325955,0.010010
HA,0.000000,NaN,NaN,59.973924,NaN,NaN


### Twenty years rolling window estimation scheme

In [8]:
## Out-of-sample: 1957:01-2020:12
in_out_1957 = predictor_df.index[predictor_df['month'] == 195701][0]
actual_1957 = actual[in_out_1957:, ]
y_pred_HA_1957 = y_pred_HA[in_out_1957:, ]
MSFE_HA_1957 = mean_squared_error(y_pred_HA_1957, actual_1957)

# Machine Learning methods used in GKX (2020)
y_pred_PLS_1957, y_pred_PCR_1957,  y_pred_LASSO_1957 = [], [], []
y_pred_ENet_1957, y_pred_RF_1957 = [], []
y_pred_NN2_1957, y_pred_NN4_1957 = [], []

## Other commonly used machine learning method
y_pred_Ridge_1957 = []

# control the update month of models during out-of-sample period. 
month_index = 1  # We update our models annually, meaning we refresh them in months 1, 13, 25, ...


for t in tqdm(range(in_out_1957, N)):
    #
    X_train_all = predictor[(t - 20 * 12):t, 1:n_cols]
    y_train_all = predictor[(t - 20 * 12):t, 0]
    # set 15% of all the train data as validation set
    X_train = X_train_all[0:int(len(X_train_all) * 0.85), :]
    X_validation = X_train_all[int(len(X_train_all) * 0.85):t, :]
    y_train = y_train_all[0:int(len(X_train_all) * 0.85)]
    y_validation = y_train_all[int(len(X_train_all) * 0.85):t]
    #
    if month_index % 12 == 1:
        month_index += 1

        # PLS
        PLS_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PLS_result = {}
        for param in ParameterGrid(PLS_param):
            PLS = PLSRegression(**param)
            PLS.fit(X_train, y_train)
            mse = mean_squared_error(PLS.predict(X_validation), y_validation)
            PLS_result[str(param)] = mse

        PLS_best_param = eval(min(PLS_result, key=PLS_result.get))
        PLS_model = PLSRegression(**PLS_best_param)
        PLS_model.fit(X_train_all, y_train_all)
        y_pred_PLS_1957.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])

        # PCR
        PCR_param = {'n_components': [1, 2, 3, 4, 5, 6, 7, 8]}
        PCR_result = {}
        for param in ParameterGrid(PCR_param):
            pca = PCA(**param)
            pca.fit(X_train)
            comps = pca.transform(X_train)
            forecast = LinearRegression()
            forecast.fit(comps, y_train)
            mse = mean_squared_error(forecast.predict(pca.transform(X_validation)), y_validation)
            PCR_result[str(param)] = mse
        #
        PCR_best_param = eval(min(PCR_result, key=PCR_result.get))
        #
        PCR_model = PCA(**PCR_best_param)
        PCR_model.fit(X_train_all)
        PCR_comps = PCR_model.transform(X_train_all)
        PCR_forecast = LinearRegression()
        PCR_forecast.fit(PCR_comps, y_train_all)
        y_pred_PCR_1957.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])

        # LASSO
        LASSO_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2))}
        LASSO_result = {}
        for param in ParameterGrid(LASSO_param):
            LASSO = Lasso(**param)
            LASSO.fit(X_train, y_train)
            mse = mean_squared_error(LASSO.predict(X_validation), y_validation)
            LASSO_result[str(param)] = mse
        #
        LASSO_best_param = eval(min(LASSO_result, key=LASSO_result.get))
        #
        LASSO_model = Lasso(**LASSO_best_param)
        LASSO_model.fit(X_train_all, y_train_all)
        y_pred_LASSO_1957.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])

        # ENet
        ENet_param = {'alpha': list(10 ** np.arange(-4, 1 + 0.001, 0.2)),
                      'l1_ratio': list(np.arange(0.2, 1, 0.3))}
        ENet_result = {}
        for param in ParameterGrid(ENet_param):
            ENet = ElasticNet(**param)
            ENet.fit(X_train, y_train)
            mse = mean_squared_error(ENet.predict(X_validation), y_validation)
            ENet_result[str(param)] = mse

        ENet_best_param = eval(min(ENet_result, key=ENet_result.get))
        ENet_model = ElasticNet(**ENet_best_param)
        ENet_model.fit(X_train_all, y_train_all)
        y_pred_ENet_1957.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])


        # RF
        RF_param = {'n_estimators': [10, 50, 100, 150, 200],
                    'max_depth': [2, 3, 4],
                    'min_samples_leaf': [1, 3, 5]}
        RF_result = {}
        for param in ParameterGrid(RF_param):
            RF = RandomForestRegressor(**param)
            RF.fit(X_train, y_train)
            mse = mean_squared_error(RF.predict(X_validation), y_validation)
            RF_result[str(param)] = mse

        RF_best_param = eval(min(RF_result, key=RF_result.get))
        RF_model = RandomForestRegressor(**RF_best_param)
        RF_model.fit(X_train_all, y_train_all)
        y_pred_RF_1957.append(RF_model.predict(predictor[[t], 1:n_cols])[0])

        # Neural Network Models: NN2 & NN4
        X_train_all_tensor = torch.tensor(X_train_all, dtype=torch.float)
        y_train_all_tensor = torch.tensor(y_train_all.reshape(-1, 1), dtype=torch.float)
        X_train_tensor = torch.tensor(X_train, dtype=torch.float)
        y_train_tensor = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float)
        X_validation_tensor = torch.tensor(X_validation, dtype=torch.float)
        y_validation_tensor = torch.tensor(y_validation.reshape(-1, 1), dtype=torch.float)

        # NN2
        NN2_result = {}
        NN2_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_output": 1}
        NN2_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                    'lr': [0.001, 0.01],
                    'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN2_param):
            NN2 = NeuralNetRegressor(Net2, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN2_architecture, **param)
            NN2.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN2.predict(X_validation_tensor), y_validation)
            NN2_result[str(param)] = mse
        
        #
        NN2_best_param = eval(min(NN2_result, key=NN2_result.get))
        NN2_model = NeuralNetRegressor(Net2, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN2_architecture, **NN2_best_param)
        NN2_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN2_1957.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        #


        # NN4
        NN4_result = {}
        NN4_architecture = {"module__n_feature": X_train_tensor.shape[1],
                            "module__n_hidden1": 32, "module__n_hidden2": 16,
                            "module__n_hidden3": 8,  "module__n_hidden4": 4,
                            "module__n_output": 1}
        NN4_param = {'module__dropout': [0.2, 0.4, 0.6, 0.8],
                     'lr': [0.001, 0.01],
                     'optimizer__weight_decay': [0.1, 0.01, 0.001]}
        for param in ParameterGrid(NN4_param):
            NN4 = NeuralNetRegressor(Net4, verbose=0, max_epochs=200,
                                     optimizer=torch.optim.SGD,
                                     **NN4_architecture, **param)
            NN4.fit(X_train_tensor, y_train_tensor)
            mse = mean_squared_error(NN4.predict(X_validation_tensor), y_validation)
            NN4_result[str(param)] = mse

        #
        NN4_best_param = eval(min(NN4_result, key=NN4_result.get))
        NN4_model = NeuralNetRegressor(Net4, verbose=0, max_epochs=200, optimizer=torch.optim.SGD,
                                       **NN4_architecture, **NN4_best_param)
        NN4_model.fit(X_train_all_tensor, y_train_all_tensor)
        y_pred_NN4_1957.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])


        ## Other commmonly used ML methods
        # Ridge
        Ridge_param = {'alpha': list(10 ** np.arange(0, 20 + 0.001, 1))}
        Ridge_result = {}
        for param in ParameterGrid(Ridge_param):
            RIDGE = Ridge(**param)
            RIDGE.fit(X_train, y_train)
            mse = mean_squared_error(RIDGE.predict(X_validation), y_validation)
            Ridge_result[str(param)] = mse
        #
        Ridge_best_param = eval(min(Ridge_result, key=Ridge_result.get))
        Ridge_model = Ridge(**Ridge_best_param)
        Ridge_model.fit(X_train_all, y_train_all)
        y_pred_Ridge_1957.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

    else:
        month_index += 1
        y_pred_PLS_1957.append(PLS_model.predict(predictor[[t], 1:n_cols])[0][0])
        y_pred_PCR_1957.append(PCR_forecast.predict(PCR_model.transform(predictor[[t], 1:n_cols]))[0])
        y_pred_LASSO_1957.append(LASSO_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_ENet_1957.append(ENet_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_RF_1957.append(RF_model.predict(predictor[[t], 1:n_cols])[0])
        y_pred_NN2_1957.append(NN2_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        y_pred_NN4_1957.append(NN4_model.predict(torch.tensor(predictor[[t], 1:n_cols], dtype=torch.float))[0][0])
        # Other commonly used ML methods
        y_pred_Ridge_1957.append(Ridge_model.predict(predictor[[t], 1:n_cols])[0])

        

        
y_ml_pred_1957 = pd.DataFrame(np.array([y_pred_PLS_1957, y_pred_PCR_1957, y_pred_LASSO_1957,
                           y_pred_ENet_1957, y_pred_RF_1957, y_pred_NN2_1957,  
                           y_pred_NN4_1957, y_pred_Ridge_1957]),
                 index=['PLS', 'PCR', 'LASSO', 'ENet', 
                        'RF', 'NN2', 'NN4', 'Ridge'],
                 columns=predictor_df.month[in_out_1957:N]).T

# Performance compared with HA benchmark

ml_oos_performance_20years = []

for col in y_ml_pred_1957.columns:
    oos_r_square = compute_oos_r_square(actual_1957, y_pred_HA_1957, y_ml_pred_1957[[col]].to_numpy())
    MSFE_adjusted, pvalue_MSFE = CW_test(actual_1957, y_pred_HA_1957, y_ml_pred_1957[[col]].to_numpy())
    success_ratio, PT_stat, pvalue_PT = PT_test(actual_1957, y_ml_pred_1957[[col]].to_numpy())
    ml_oos_performance_20years.append([oos_r_square * 100, MSFE_adjusted, pvalue_MSFE, success_ratio * 100, PT_stat, pvalue_PT])


ml_oos_performance_df_20years = pd.DataFrame(np.array(ml_oos_performance_20years),
                                          index=y_ml_pred_1957.columns,
                                          columns=['oos_r_square', 'MSFE_adjusted', 'pvalue_MSFE',
                                                   'success_ratio', 'PT_stat', 'pvalue_PT'])
ml_oos_performance_df_20years.loc['HA'] = [0, np.nan, np.nan, success_ratio_HA_1957 * 100, PT_HA_1957, p2_HA_1957]
ml_oos_performance_df_20years

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 767/767 [59:03<00:00,  4.62s/it]


,oos_r_square,MSFE_adjusted,pvalue_MSFE,success_ratio,PT_stat,pvalue_PT
PLS,-9.845205,0.158533,0.437018,54.498044,1.519034,0.064377
PCR,-4.371883,0.794236,0.213529,58.539765,2.490715,0.006374
LASSO,-3.552983,0.877620,0.190075,56.453716,1.127861,0.129689
ENet,-4.274418,0.664717,0.253116,57.887875,2.087509,0.018421
RF,-16.130001,-0.308727,0.621235,54.367666,0.120474,0.452054
NN2,-30.501839,0.497311,0.309485,56.062581,1.102496,0.135123
NN4,-76.556965,-1.000122,0.841374,54.889179,-0.584516,0.720563
Ridge,-2.587061,1.139190,0.127312,58.018253,1.805065,0.035532
HA,0.000000,NaN,NaN,59.973924,NaN,NaN


In [9]:
ml_oos_performance_rolling_window = pd.concat([ml_oos_performance_df_5years, ml_oos_performance_df_10years, ml_oos_performance_df_20years], axis=1)


with pd.ExcelWriter("ml_equity_premium_robust_checks.xlsx") as writer:
    ml_oos_performance_rolling_window.to_excel(writer, sheet_name='rolling_window_estimation')

ml_oos_performance_rolling_window

,oos_r_square,MSFE_adjusted,pvalue_MSFE,success_ratio,PT_stat,pvalue_PT,oos_r_square,MSFE_adjusted,pvalue_MSFE,success_ratio,PT_stat,pvalue_PT,oos_r_square,MSFE_adjusted,pvalue_MSFE,success_ratio,PT_stat,pvalue_PT
PLS,-47.239596,1.083377,0.139321,52.411995,-0.164357,0.565275,-15.893212,1.049371,0.147004,55.410691,1.356501,0.087470,-9.845205,0.158533,0.437018,54.498044,1.519034,0.064377
PCR,-11.831484,0.818648,0.206494,56.062581,1.013248,0.155471,-6.902878,0.899997,0.184061,57.757497,1.715461,0.043130,-4.371883,0.794236,0.213529,58.539765,2.490715,0.006374
LASSO,-82.461558,-0.003547,0.501415,57.105606,1.118965,0.131578,-4.771582,1.970124,0.024412,59.843546,2.558406,0.005258,-3.552983,0.877620,0.190075,56.453716,1.127861,0.129689
ENet,-74.396652,0.606137,0.272212,55.410691,0.585414,0.279135,-5.346111,1.982762,0.023697,59.843546,2.747300,0.003004,-4.274418,0.664717,0.253116,57.887875,2.087509,0.018421
RF,-16.913349,0.194882,0.422743,52.542373,-0.226891,0.589746,-14.836244,0.534075,0.296645,51.368970,-0.222107,0.587885,-16.130001,-0.308727,0.621235,54.367666,0.120474,0.452054
NN2,-102.982502,-0.304749,0.619721,46.675359,-2.474727,0.993333,-100.028150,0.707580,0.239603,52.542373,0.229788,0.409128,-30.501839,0.497311,0.309485,56.062581,1.102496,0.135123
NN4,-358.211154,-0.810580,0.791197,51.499348,-1.006715,0.842964,-103.312912,-1.065146,0.856595,55.541069,1.161620,0.122695,-76.556965,-1.000122,0.841374,54.889179,-0.584516,0.720563
Ridge,-26.633373,0.143339,0.443011,56.453716,0.790713,0.214556,-5.562798,1.265462,0.102853,58.930900,2.325955,0.010010,-2.587061,1.139190,0.127312,58.018253,1.805065,0.035532
HA,0.000000,NaN,NaN,59.973924,NaN,NaN,0.000000,NaN,NaN,59.973924,NaN,NaN,0.000000,NaN,NaN,59.973924,NaN,NaN
